<a href="https://colab.research.google.com/github/andreldeod/portfolio/blob/main/resources/code/ArcGIS_REST_Server/download_raster_from_REST_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code to download peat rasters from ArcGIS REST Server

By André

First chunk is for running on colab. Second is the same code but made to run locally.

Code is to download Forest Service peat rasters:

[histosol_top20crops](https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/histosol_top20crops/ImageServer)

[histosol_percents](https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/histosol_percents/ImageServer)

# Download data for N Carolina extent (Run on Google COLAB)

In [ ]:
# Ensure necessary libraries are installed and suppress output
!pip install requests > /dev/null
!pip install rasterio > /dev/null

from google.colab import drive

# Mount Google Drive to access your files
drive.mount('/content/drive')

In [ ]:
import requests
import os
import math
import rasterio
from rasterio.merge import merge
from google.colab import drive


# Define the path to your Google Drive directory
output_directory = "/content/drive/My Drive/TerraCarbon/Code_Andre/peat"
tiles_directory = os.path.join(output_directory, "tiles")

# Data name to download
name = "histosol_percents"

# Define the URL for the ArcGIS REST service
url = f"https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/{name}/ImageServer/exportImage"

# Define the bounding box for North Carolina (in EPSG:3857)
bbox = {
    "xmin": -9000000,  # approximate xmin for NC
    "ymin": 3830000,   # approximate ymin for NC
    "xmax": -8100000,  # approximate xmax for NC
    "ymax": 4500000    # approximate ymax for NC
}

# Pixel size
pixel_size = 30

# Tile size constraints (in pixels)
tile_width = 15000
tile_height = 4100

# Calculate the width and height in meters
tile_width_m = tile_width * pixel_size
tile_height_m = tile_height * pixel_size

# Calculate the number of tiles needed
width = bbox["xmax"] - bbox["xmin"]
height = bbox["ymax"] - bbox["ymin"]
tiles_x = math.ceil(width / tile_width_m)
tiles_y = math.ceil(height / tile_height_m)

# Ensure the directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(tiles_directory, exist_ok=True)

# List to store file paths of downloaded tiles
tile_paths = []

# Loop over tiles and download each one
for i in range(tiles_x):
    for j in range(tiles_y):
        xmin = bbox["xmin"] + i * tile_width_m
        xmax = min(xmin + tile_width_m, bbox["xmax"])
        ymin = bbox["ymin"] + j * tile_height_m
        ymax = min(ymin + tile_height_m, bbox["ymax"])

        # Calculate the actual tile size in pixels for the request
        actual_tile_width = int((xmax - xmin) / pixel_size)
        actual_tile_height = int((ymax - ymin) / pixel_size)

        # Define the parameters for the exportImage request
        params = {
            "bbox": f"{xmin},{ymin},{xmax},{ymax}",
            "bboxSR": 102100,
            "size": f"{actual_tile_width},{actual_tile_height}",
            "imageSR": 102100,
            "format": "tiff",
            "f": "image"
        }

        # Send the GET request to the server
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Define the output path for the GeoTIFF tile
            tile_path = os.path.join(tiles_directory, f"tile_{i}_{j}.tiff")
            # Save the GeoTIFF file to the specified directory
            with open(tile_path, 'wb') as file:
                file.write(response.content)
            tile_paths.append(tile_path)
            print(f"Tile ({i}, {j}) successfully downloaded and saved as '{tile_path}'")
        else:
            print(f"Failed to download tile ({i}, {j}). HTTP status code: {response.status_code}")
            print(f"Response content: {response.content}")

# Merge tiles into a single image
if tile_paths:
    src_files_to_mosaic = [rasterio.open(fp) for fp in tile_paths]
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Define the metadata for the merged image
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Save the merged image
    merged_output_path = os.path.join(output_directory, f"{name}_north_carolina.tiff")
    with rasterio.open(merged_output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    print(f"Merged GeoTIFF successfully saved as '{merged_output_path}'")

    # Clean up by closing all tile files and deleting them
    for src in src_files_to_mosaic:
        src.close()
    for tile_path in tile_paths:
        os.remove(tile_path)
    print("All individual tile files have been deleted.")

Tile (0, 0) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_0.tiff'
Tile (0, 1) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_1.tiff'
Tile (0, 2) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_2.tiff'
Tile (0, 3) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_3.tiff'
Tile (0, 4) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_4.tiff'
Tile (0, 5) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_0_5.tiff'
Tile (1, 0) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_1_0.tiff'
Tile (1, 1) successfully downloaded and saved as '/content/drive/My Drive/TerraCarbon/Code_Andre/tiles/tile_1_1.tiff'
Tile (1, 2) successfully downloaded and saved as '/conte

# Download data for N Carolina extent (Run locally)

In [ ]:
%% capture
pip install rasterio

In [ ]:
import requests
import os
import math
import rasterio
from rasterio.merge import merge

# This script downloads raster data for North Carolina as smaller GeoTIFF tiles
# and merges them into a single GeoTIFF file.

# Data name to download
#name = "histosol_top20crops"
name = "histosol_percents"

# Define the URL for the ArcGIS REST service
url = f"https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/{name}/ImageServer/exportImage"

# Define the bounding box for North Carolina (in EPSG:3857)
bbox = {
    "xmin": -9000000,  # approximate xmin for NC
    "ymin": 3830000,   # approximate ymin for NC
    "xmax": -8100000,  # approximate xmax for NC
    "ymax": 4500000    # approximate ymax for NC
}

# Pixel size
pixel_size = 30

# Tile size constraints (in pixels)
tile_width = 15000
tile_height = 4100

# Calculate the width and height in meters
tile_width_m = tile_width * pixel_size
tile_height_m = tile_height * pixel_size

# Calculate the number of tiles needed
width = bbox["xmax"] - bbox["xmin"]
height = bbox["ymax"] - bbox["ymin"]
tiles_x = math.ceil(width / tile_width_m)
tiles_y = math.ceil(height / tile_height_m)

# Directory where you want to save the tiles and the merged image
output_directory = "C:\\TerraCarbon\\peat"
tiles_directory = os.path.join(output_directory, "tiles")

# Ensure the directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(tiles_directory, exist_ok=True)

# List to store file paths of downloaded tiles
tile_paths = []

# Loop over tiles and download each one
for i in range(tiles_x):
    for j in range(tiles_y):
        xmin = bbox["xmin"] + i * tile_width_m
        xmax = min(xmin + tile_width_m, bbox["xmax"])
        ymin = bbox["ymin"] + j * tile_height_m
        ymax = min(ymin + tile_height_m, bbox["ymax"])

        # Calculate the actual tile size in pixels for the request
        actual_tile_width = int((xmax - xmin) / pixel_size)
        actual_tile_height = int((ymax - ymin) / pixel_size)

        # Define the parameters for the exportImage request
        params = {
            "bbox": f"{xmin},{ymin},{xmax},{ymax}",
            "bboxSR": 102100,
            "size": f"{actual_tile_width},{actual_tile_height}",
            "imageSR": 102100,
            "format": "tiff",
            "f": "image"
        }

        # Send the GET request to the server
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Define the output path for the GeoTIFF tile
            tile_path = os.path.join(tiles_directory, f"tile_{i}_{j}.tiff")
            # Save the GeoTIFF file to the specified directory
            with open(tile_path, 'wb') as file:
                file.write(response.content)
            tile_paths.append(tile_path)
            print(
                f"Tile ({i}, {j}) successfully downloaded and saved as '{tile_path}'")
        else:
            print(
                f"Failed to download tile ({i}, {j}). HTTP status code: {response.status_code}")
            print(f"Response content: {response.content}")

# Merge tiles into a single image
if tile_paths:
    src_files_to_mosaic = [rasterio.open(fp) for fp in tile_paths]
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Define the metadata for the merged image
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Save the merged image
    merged_output_path = os.path.join(
        output_directory, f"{name}_north_carolina.tiff")
    with rasterio.open(merged_output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    print(f"Merged GeoTIFF successfully saved as '{merged_output_path}'")

    # Clean up by closing all tile files and deleting them
    for src in src_files_to_mosaic:
        src.close()
    for tile_path in tile_paths:
        os.remove(tile_path)
    print("All individual tile files have been deleted.")

# Download data for full extent (Run locally)

In [ ]:
import requests
import os
import math
import rasterio
from rasterio.merge import merge
from time import sleep
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# This script downloads raster data for a specified extent as smaller GeoTIFF
# tiles, merges them into a single GeoTIFF file, and deletes the individual
# tiles.

# Data name to download
#name = "histosol_top20crops"
name = "histosol_percents"

# Define the URL for the ArcGIS REST service
url = f"https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/{name}/ImageServer/exportImage"

# Define the bounding box for the specified extent (in EPSG:3857)
bbox = {
    "xmin": -14236365,  # xmin for the full extent
    "ymin": 2624775,    # ymin for the full extent
    "xmax": -7274205,   # xmax for the full extent
    "ymax": 6728835     # ymax for the full extent
}

# Pixel size
pixel_size = 30

# Tile size constraints (in pixels)
tile_width = 15000
tile_height = 4100

# Calculate the width and height in meters
tile_width_m = tile_width * pixel_size
tile_height_m = tile_height * pixel_size

# Calculate the number of tiles needed
width = bbox["xmax"] - bbox["xmin"]
height = bbox["ymax"] - bbox["ymin"]
tiles_x = math.ceil(width / tile_width_m)
tiles_y = math.ceil(height / tile_height_m)

# Directory where you want to save the tiles and the merged image
output_directory = "C:\\TerraCarbon\\peat"
tiles_directory = os.path.join(output_directory, "tiles")

# Ensure the directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(tiles_directory, exist_ok=True)

# List to store file paths of downloaded tiles
tile_paths = []

# Setup retry strategy
retry_strategy = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)

# Loop over tiles and download each one
for i in range(tiles_x):
    for j in range(tiles_y):
        xmin = bbox["xmin"] + i * tile_width_m
        xmax = min(xmin + tile_width_m, bbox["xmax"])
        ymin = bbox["ymin"] + j * tile_height_m
        ymax = min(ymin + tile_height_m, bbox["ymax"])

        # Calculate the actual tile size in pixels for the request
        actual_tile_width = int((xmax - xmin) / pixel_size)
        actual_tile_height = int((ymax - ymin) / pixel_size)

        # Define the parameters for the exportImage request
        params = {
            "bbox": f"{xmin},{ymin},{xmax},{ymax}",
            "bboxSR": 102100,
            "size": f"{actual_tile_width},{actual_tile_height}",
            "imageSR": 102100,
            "format": "tiff",
            "f": "image"
        }

        # Send the GET request to the server
        try:
            response = http.get(url, params=params, timeout=120)
            response.raise_for_status()  # Raise an exception for HTTP errors

            # Define the output path for the GeoTIFF tile
            tile_path = os.path.join(tiles_directory, f"tile_{i}_{j}.tiff")
            # Save the GeoTIFF file to the specified directory
            with open(tile_path, 'wb') as file:
                file.write(response.content)
            tile_paths.append(tile_path)
            print(
                f"Tile ({i}, {j}) successfully downloaded and saved as '{tile_path}'")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download tile ({i}, {j}). Error: {e}")
            print(
                f"Response content: {response.content if response else 'No response content'}")

# Merge tiles into a single image
if tile_paths:
    src_files_to_mosaic = [rasterio.open(fp) for fp in tile_paths]
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Define the metadata for the merged image
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Save the merged image
    merged_output_path = os.path.join(output_directory, f"{name}.tiff")
    with rasterio.open(merged_output_path, "w", **out_meta) as dest:
        dest.write(mosaic)
    print(f"Merged GeoTIFF successfully saved as '{merged_output_path}'")

    # Clean up by closing all tile files and deleting them
    for src in src_files_to_mosaic:
        src.close()
    for tile_path in tile_paths:
        os.remove(tile_path)
    print("All individual tile files have been deleted.")


# Print class names (Run locally or on COLAB)

In [ ]:
import requests

# Data name to download
name = "histosol_top20crops"
#name = "histosol_percents"


# Define the URL for the Legend operation on the ArcGIS REST service
url = f"https://apps.fs.usda.gov/fsgisx03/rest/services/wo_nfs_gtac/{name}/ImageServer/legend"

# Define the parameters for the Legend request
params = {
    "f": "json"
}

# Send the GET request to the server
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    if 'layers' in data:
        layers = data['layers']
        for layer in layers:
            print(f"Layer Name: {layer['layerName']}")
            for legend in layer['legend']:
                value = legend['values']
                class_name = legend['label']
                print(f"Value: {value}, Class Name: {class_name}")
    else:
        print("No layers found in the response.")
else:
    print(f"Failed to get legend. HTTP status code: {response.status_code}")
    print(f"Response content: {response.content}")


Layer Name: wo_nfs_gtac/histosol_top20crops
Value: ['Corn'], Class Name: Corn
Value: ['Soybeans'], Class Name: Soybeans
Value: ['Spring Wheat'], Class Name: Spring Wheat
Value: ['Winter Wheat'], Class Name: Winter Wheat
Value: ['Oats'], Class Name: Oats
Value: ['Alfalfa'], Class Name: Alfalfa
Value: ['Hay/Non Alfalfa'], Class Name: Hay/Non Alfalfa
Value: ['Other Crops'], Class Name: Other Crops
Value: ['Sugarcane'], Class Name: Sugarcane
Value: ['Onions'], Class Name: Onions
Value: ['Sod/Grass Seed'], Class Name: Sod/Grass Seed
Value: ['Fallow'], Class Name: Fallow
Value: ['Open Water'], Class Name: Open Water
Value: ['Developed'], Class Name: Developed
Value: ['Grassland/Pasture'], Class Name: Grassland/Pasture
Value: ['Herb Wetlands'], Class Name: Herb Wetlands
Value: ['Woody Wetlands'], Class Name: Woody Wetlands
Value: ['Oranges'], Class Name: Oranges
Value: ['Blueberries'], Class Name: Blueberries
Value: ['Cranberries'], Class Name: Cranberries
